# Convolutional Autoencoder (CAE)
A Convolutional Autoencoder (CAE) is a type of deep neural network used primarily for unsupervised learning tasks, such as image reconstruction and feature learning. It combines the principles of convolutional neural networks (CNNs) and autoencoders to efficiently encode and decode data, particularly images.
Key Components of a Convolutional Autoencoder:
### Encoder:
* **Convolutional Layers:** The encoder uses convolutional layers to capture spatial hierarchies in the input data. Convolutional layers apply filters to the input, generating feature maps that highlight different aspects of the data.
* **Pooling Layers:** Often, pooling layers (e.g., max pooling) are used to downsample the feature maps, reducing their dimensions and retaining essential features.
* **Latent Space Representation:** The encoder compresses the input data into a lower-dimensional latent space representation. This compressed form captures the most important information of the input data.
### Decoder:
* **Transposed Convolutional Layers:** The decoder mirrors the encoder but uses transposed convolutional (or deconvolutional) layers to upsample the latent representation back to the original input size.
* **Reconstruction:** The decoder reconstructs the input data from its compressed form, ideally producing an output that is as close as possible to the original input.

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, losses

from Models import CNN_AE_vx01 as ae
ae_version = ae.version
ae_layers = ae.nlayers

In [2]:
from platform import python_version
import sys
print('Python: ' + python_version()) # Python: 3.10.9
print('numpy: ' + np.__version__) # numpy: 1.23.5
print ('tensorflow: ' + sys.modules["tensorflow"].__version__) # tensorflow: 2.10.0

Python: 3.10.9
numpy: 1.23.5
tensorflow: 2.10.0


# Initialization of the Autoencoder

In [3]:
# autoencoder = CNN_AE()
autoencoder = ae.CNN_AE()
autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())

In [4]:
autoencoder.build((None,182, 362, ae_layers))
autoencoder.encoder.summary()
autoencoder.decoder.summary()
autoencoder.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 90, 180, 16)       160       
                                                                 
 conv2d_1 (Conv2D)           (None, 45, 90, 16)        2320      
                                                                 
 conv2d_2 (Conv2D)           (None, 45, 90, 1)         145       
                                                                 
Total params: 2,625
Trainable params: 2,625
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_transpose (Conv2DTra  (None, 90, 180, 16)      160       
 nspose)                                                         
                                

# Load autoencoder from weights

In [3]:

version = "34"
autoencoder = ae.CNN_AE()
autoencoder.encoder.load_weights('./Weights/cnn_ae_v' + ae_version + '_encoder_weights_' + version)
autoencoder.decoder.load_weights('./Weights/cnn_ae_v' + ae_version + '_decoder_weights_' + version)
autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())
autoencoder.build((None,182, 362, ae_layers))

# Load training dataset

In [8]:
if ae_layers == 1:
    datacube_precip = np.load("../data/WaterPrecip_datacube_CAE_single.npy")
else:
    datacube_precip = np.load("../data/WaterPrecip_datacube_CNN_x.npy")
    
datacube_precip.shape

(9232, 182, 362, 1)

In [9]:
nTest = 1000
iTest = np.random.choice(datacube_precip.shape[0],nTest, replace=False)
iTrain = [i for i in range(datacube_precip.shape[0]) if i not in iTest]
datacube_precip[iTest,1:181,1:361,:].shape

(1000, 180, 360, 1)

# Train Autoencoder

In [23]:
autoencoder.fit(datacube_precip[iTrain,:,:,:ae_layers], datacube_precip[iTrain,1:181,1:361,:ae_layers],
               epochs=10,
               shuffle=True,
               validation_data=(datacube_precip[iTest,:,:,:ae_layers], datacube_precip[iTest,1:181,1:361,:ae_layers]))

Epoch 1/10
258/258 [==============================] - 84s 327ms/step - loss: 3.9944 - val_loss: 4.0214
Epoch 2/10
258/258 [==============================] - 82s 316ms/step - loss: 3.9875 - val_loss: 4.0233
Epoch 3/10
258/258 [==============================] - 87s 338ms/step - loss: 3.9820 - val_loss: 4.0081
Epoch 4/10
258/258 [==============================] - 84s 325ms/step - loss: 3.9745 - val_loss: 4.0022
Epoch 5/10
258/258 [==============================] - 88s 341ms/step - loss: 3.9706 - val_loss: 3.9976
Epoch 6/10
258/258 [==============================] - 79s 305ms/step - loss: 3.9632 - val_loss: 3.9903
Epoch 7/10
258/258 [==============================] - 79s 306ms/step - loss: 3.9592 - val_loss: 3.9856
Epoch 8/10
258/258 [==============================] - 83s 321ms/step - loss: 3.9551 - val_loss: 3.9821
Epoch 9/10
258/258 [==============================] - 80s 312ms/step - loss: 3.9497 - val_loss: 3.9778
Epoch 10/10
258/258 [==============================] - 84s 325ms/step - l

# Save weights

In [25]:
version = "43"
autoencoder.encoder.save_weights('./Weights/cnn_ae_v' + ae_version + '_encoder_weights_' + version)
autoencoder.decoder.save_weights('./Weights/cnn_ae_v' + ae_version + '_decoder_weights_' + version)